In [1]:
# %load mnist_loader.py

#### Libraries
# Standard library
import pickle 
##Esta libreria nos permite serializar objetos es decir convertir un objeto en base a 
#un lenguaje específico y deserializar que es el proceso análogo de string a objeto)
import gzip ##para comprimir y descomprimir archivo.zip como el mnist.pkl.gz

# Third-party libraries
import numpy as np## nos permite realizar loscalculos con arrays

def load_data(): ##vamos a definir una función 

    f = gzip.open('mnist.pkl.gz', 'rb') #abre el archivo.zip, el modo rb, predeterminado como lectura de datos binarios) 
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")#definimos las variables a las que con
    #.load vamos a deserealizar y a dodificar en ascii global
    f.close()# cierra el mnist.pk
    return (training_data, validation_data, test_data)
#regresa las variables en datos de entrenamiento, datos de validación, y de prueba 

def load_data_wrapper():## pre procesamiento de los datos

    tr_d, va_d, te_d = load_data()##a estas variables les aplicamos la función
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]] ##reformamos una matriz de 784 x1 para la lista tr_d
    #comenzando por el elemento 0
    training_results = [vectorized_result(y) for y in tr_d[1]]##vamos a definir antes la función 
    training_data = zip(training_inputs, training_results)##como un ziper de una chamrra retorna un nuevo iterable 
    #cuyos elementos son tuplas que contienen un elemento de training data con otro de training results
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])#proceso análogo al zip para training data
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)#regresa los nuervos arrayas que formamos en la función.

def vectorized_result(j):##esta función toma un entero 
    e = np.zeros((10, 1))#matriz de ceros de 10x1
    e[j] = 1.0
    return e


In [2]:
# %load network.py



#### Libraries
# Standard library
import random # está libreria nos permite obtener datos aleatorios para alimentar 

# Third-party libraries
import numpy as np

class Network(object):##definimos la clase que será la neurona

    def __init__(self, sizes):##es la función con los parametros no de neuronas por capas y pesos

        self.num_layers = len(sizes)#tamaño de capas
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)#esta funcion crea los pesos aleatoriamente
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):#valor de inicializacion por función sigmoide
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)##producto punto y le sumamos la bias
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,##obtiene los pesos para el backpropagation y evita que se atore
            test_data=None):


        training_data = list(training_data)##lista de datos para entrenar
        n = len(training_data)#nos da el tamaño de training data

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)##reorganizamos aleatoriamente los datos
            mini_batches = [
                training_data[k:k+mini_batch_size]#reparte los datos entre el númeroi de minibatches
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)##el.update le actualiza al minibatch
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test))#este ciclo nos permite visualizar 
                #datos o epocas
            else:
                print("Epoch {} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):#la función que actualiza el minibatch
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]#llena de ceros
        #las listas que definimos se van llenando de la suma del grad por minibatch
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)#funcion de coste
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]#A los nuevos datos que obtuvimos del 
        #grad del minibatch  se les resta el valor del aprendizaje
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # lista de todas las activaciones de capa en capa
        zs = [] # definimos una lista a la que le vamos a meter los pesos por capa
        for b, w in zip(self.biases, self.weights):#ciclo para  ir juntando pesos y bias por capa
            z = np.dot(w, activation)+b
            zs.append(z)#añade
            activation = sigmoid(z)#aplica la función sigmoide a mi producto punto antes definido
            activations.append(activation)#añade a activations cada valor previamente calculado
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])#calcula el error de atrás hacía adelante es decir 
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
    for l in range(2, self.num_layers):#errores para el gradiente 
            z = zs[-l]
            sp = sigmoid_prime(z)#derivada
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):

        test_results = [(np.argmax(self.feedforward(x)), y)#función para argumento mayor de la función feedforward para calcular 
                        #la salida de la red
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)#regersa la suma

    def cost_derivative(self, output_activations, y):##funcion de costo en relacion con las salidas
      
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))##función sigmoide

def sigmoid_prime(z):##derivada de la función sigmoide
    return sigmoid(z)*(1-sigmoid(z))

In [5]:
import mnist_loader ## importar código/bloque de los datos

training_data, validation_data, test_data = mnist_loader.load_data_wrapper() 
training_data = list(training_data) 
import network #importar código de la red
 
net = network.Network([784, 30, 10]) ##parametros para entrenar
net.SGD(training_data, 55, 10, 0.07, test_data=test_data)

Epoch 0 : 4649 / 10000
Epoch 1 : 5858 / 10000
Epoch 2 : 6315 / 10000
Epoch 3 : 6519 / 10000
Epoch 4 : 6690 / 10000
Epoch 5 : 7171 / 10000
Epoch 6 : 7542 / 10000
Epoch 7 : 7641 / 10000
Epoch 8 : 7718 / 10000
Epoch 9 : 7788 / 10000
Epoch 10 : 7859 / 10000
Epoch 11 : 7886 / 10000
Epoch 12 : 7935 / 10000
Epoch 13 : 7966 / 10000
Epoch 14 : 7993 / 10000
Epoch 15 : 8023 / 10000
Epoch 16 : 8054 / 10000
Epoch 17 : 8065 / 10000
Epoch 18 : 8090 / 10000
Epoch 19 : 8100 / 10000
Epoch 20 : 8106 / 10000
Epoch 21 : 8115 / 10000
Epoch 22 : 8128 / 10000
Epoch 23 : 8143 / 10000
Epoch 24 : 8149 / 10000
Epoch 25 : 8162 / 10000
Epoch 26 : 8176 / 10000
Epoch 27 : 8177 / 10000
Epoch 28 : 8195 / 10000
Epoch 29 : 8203 / 10000
Epoch 30 : 8208 / 10000
Epoch 31 : 8211 / 10000
Epoch 32 : 8225 / 10000
Epoch 33 : 8225 / 10000
Epoch 34 : 8236 / 10000
Epoch 35 : 8241 / 10000
Epoch 36 : 8243 / 10000
Epoch 37 : 8265 / 10000
Epoch 38 : 8266 / 10000
Epoch 39 : 8268 / 10000
Epoch 40 : 8264 / 10000
Epoch 41 : 8283 / 10000
Ep